# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.data.dataset_factory import TabularDatasetFactory


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
workspace = Workspace.from_config()

# choose a name for experiment
experiment_name = 'aml-stroke-experiment'
project_folder = './aml-project'

experiment=Experiment(workspace, experiment_name)


In [3]:
found = False
key = "stroke-dataset"
description_text = "Stroke Dataset - Capstone project"

if key in workspace.datasets.keys(): 
        found = True
        dataset = workspace.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/ciccioska/Udacity-AzureML-Capstone-Project/master/healthcare-dataset-stroke-data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data) 

        #Register Dataset in Workspace
        dataset = dataset.register(workspace=workspace,
                                   name=key,
                                   description=description_text)


x_df = dataset.to_pandas_dataframe().dropna()

x_df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,1.000000


In [4]:
#prepare dataset
x_df["work_type"] = x_df.work_type.apply(lambda s: 1 if s == "Private" else 0 if s == "Self-employed" else -1)
x_df["bmi"] = x_df.bmi.apply(lambda s: 0 if s == "N/A" else s)
x_df["gender"] = x_df.gender.apply(lambda s: 1 if s == "Male" else 0)
x_df["Residence_type"] = x_df.Residence_type.apply(lambda s: 1 if s == "Urban" else 0)
x_df["smoking_status"] = x_df.smoking_status.apply(lambda s: 1 if s == "smokes" else -1 if s == "never smoked" else 2 if s == "formerly smoked" else 0)        

y_df = x_df.pop("stroke")

# Split data into train and test sets.
(x_train, x_test, y_train, y_test) = train_test_split(x_df, y_df, test_size = 0.3)
label = "stroke"

train_data_df = pd.concat([x_train, y_train], axis=1)
test_data_df = pd.concat([x_test, y_train], axis=1)

# view train dataset
train_data_df.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
208,54921,1,78.0,1,0,True,0,0,134.80,33.6,0,1
1763,66502,1,16.0,0,0,False,1,0,111.93,32.2,-1,0
3854,11692,0,53.0,0,0,False,-1,1,101.81,29.4,1,0
4067,11280,0,28.0,0,0,True,1,1,98.05,24.7,-1,0
3623,13817,1,19.0,0,0,False,1,1,123.61,25.2,0,0


In [5]:

# create a csv file to use with AutoML
if not os.path.isdir('data'): 
    os.mkdir('data')
    
# save csv file to datastore
pd.DataFrame(train_data_df).to_csv("data/dataset.csv", index=False)

# upload to datastore
datastore = workspace.get_default_datastore()
datastore.upload(src_dir='./data', target_path='classified', overwrite=True, show_progress=True)

 # datastore for AutoML
training_data = TabularDatasetFactory.from_delimited_files(path=datastore.path('classified/dataset.csv'))

Uploading an estimated of 2 files
Uploading ./data/dataset.csv
Uploaded ./data/dataset.csv, 1 files out of an estimated total of 2
Uploading ./data/.ipynb_checkpoints/dataset-checkpoint.csv
Uploaded ./data/.ipynb_checkpoints/dataset-checkpoint.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [6]:

amlcompute_cluster_name = "aml-cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=workspace, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(workspace, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.......................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
automl_settings = {
    "n_cross_validations": 2,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'norm_macro_recall',
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=training_data,
                             label_column_name="stroke",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on aml-cpu-cluster with default configuration
Running on remote compute: aml-cpu-cluster
Parent Run ID: AutoML_619c1840-31e5-46ec-b705-58c2188161d7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Generating individually featurized CV splits.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_619c1840-31e5-46ec-b705-58c2188161d7',
 'target': 'aml-cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-29T12:00:10.064383Z',
 'endTimeUtc': '2021-03-29T12:37:58.421629Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'aml-cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"aml-stroke-experiment","subscription_id":"12cd141f-7e8d-40f4-ba6e-35b327775bd4","resource_group":"ska_resources","workspace_name":"capstone_project_workspace","region":"eastus","compute_target":"aml-cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"norm_macro_recall","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_automl_run, automl_model = remote_run.get_output()

In [13]:
#Save the best model
automl_model = remote_run.register_model(model_name='stroke-experiment-model')

automl_model.download(target_dir="outputs", exist_ok=True)

'outputs/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service